**1. SLAM**  

$$ J(x) =  \frac{1}{2} \left\{ \sum_i^{nx} \| \frac{1}{\epsilon_0} (x_{i,0} - x^p_{i,0}) \| ^2+ \sum_t^{nt} \sum_i^{nx} \| \frac{1}{\epsilon_x} (x_{i,t} - F(x_{i,t-1}, x_{i-1,t-1}, x_{i+1,t-1}, x_{i,t-2})) \| ^2 + \sum_t^{nt\_obs} \sum_i^{nx\_obs} \| \frac{1}{\epsilon_y} (y_{i,t} - x_{i\_obs, t\_obs}) \| ^2 \right\} $$

**2. 4DVar**  
or Strong-constraint 4DVar. Assume model is perfect. find $\textbf{x}_0$ to result in the trajectory $M(\textbf{x}_0)$ being close to observations.  
$$ J(\textbf{x}_0) = \frac{1}{2} \left\{ (\textbf{x}_0 - \textbf{x}_0^b)^T \textbf{B}_0^{-1} (\textbf{x}_0 - \textbf{x}_0^b) + \sum_{t=0}^{nt} (\textbf{y}_t^o - H_t(M_{0 \Rightarrow t}(\textbf{x}_0)))^T \textbf{R}_t^{-1} (\textbf{y}_t^o - H_t(M_{0 \Rightarrow t}(\textbf{x}_0))) \right\} $$ 

$$ \textbf{R}_t^{-0.5} (\textbf{y}_t^o - H_t(M_{0 \Rightarrow t}(\textbf{x}_0))) $$

superscript: $b$ for background (priori), $o$ for observation  
subscript: specifies time index, $0$ indicates the begining of the window    
$\textbf{x}_0$, $\textbf{x}_0^b$: model state, vector of $n$   
$\textbf{y}_t^0$ : observation state (measured at time $t$), vector of $m_t$,   
$\textbf{B}_0$: background (priori) error covariance at time $0$, matrix of $n \times n$  
$\textbf{R}_t$: observation error covariance at time $t$, matrix of $m_t \times m_t$  
$M_{0->t}$: model integration operator ($\mathbb{R}^{n} \Rightarrow \mathbb{R}^{n}$), i.e. predict $x_t$ from $x_0$  
$H_t$: observation forward operator (($\mathbb{R}^{n} \Rightarrow \mathbb{R}^{m_t}$), i.e. project $x_t$ to observation space  
   
*Note*: $nt$ in *4DVar* can be (and usually is) different with $nt$ in $SLAM$ equation.
1. $nt$ in SLAM is number of integration steps of the assimilation window
2. $nt$ in 4DVar can be a much smaller number. Usually, for 6 hour window observations are partitioned into 7 buckets (hour of  0,1,2,3,4,5,6), which means $nt=6$. 

** Solve 4DVar**   

let $\textbf{x}_0 = \textbf{x}_0^b + \textbf{E}_0 \textbf{w}_0$,  $\textbf{E}_0$ is sqrt root of $\textbf{B}_0$ ($\textbf{B}_0 = \textbf{E}_0 {\textbf{E}_0}^T$)   for ** preconditioning + dimension-reduction**  
$\textbf{E}_0$: size $n \times p$, (use $p \ll n$ to reduce dimension in real world applications)   

$$ J(\textbf{w}_0) = \frac{1}{2} \left\{ \textbf{w}_0^T \textbf{w}_0 + \sum_{t=0}^{nt} (\textbf{y}_t^o - H_t(M_{0 \Rightarrow t}(\textbf{x}_0^b + \textbf{E}_0 \textbf{w}_0)))^T \textbf{R}_t^{-1} (\textbf{y}_t^o - H_t(M_{0 \Rightarrow t}(\textbf{x}_0^b + \textbf{E}_0 \textbf{w}_0))) \right\} $$   
    
      
** in Ceres-Solver**   
2 residual blocks:
1. $\textbf{w}_0^T \textbf{w}_0$, $residual = \textbf{w}_0$, $operator(): \mathbb{R}^p \Rightarrow \mathbb{R}^p$    
   
2. $\sum_{t=0}^{nt} (\textbf{y}_t^o - H_t(M_{0 \Rightarrow t}(\textbf{x}_0^b + \textbf{E}_0 \textbf{w}_0)))^T \textbf{R}_t^{-1} (\textbf{y}_t^o - H_t(M_{0\Rightarrow t}(\textbf{x}_0^b + \textbf{E}_0 \textbf{w}_0)))$. $operator(): \mathbb{R}^p \Rightarrow \mathbb{R}^{n\_obs}$ ($n\_obs$: total number of observations in the window)

**3. Weak-Constraint 4DVar**  
 Assume model is imperfect. find model trajectory $\textbf{x}_0 \cdots \textbf{x}_{nt}$ being close to observations, under weak-constraint of model ($M_{(t-1) \Rightarrow t}(\textbf{x}_{t-1}) \approx \textbf{x}_t$)

$$ J(\textbf{x}_0, \cdots, \textbf{x}_{nt}) = \frac{1}{2} \left\{ (\textbf{x}_0 - \textbf{x}_0^b)^T \textbf{B}_0^{-1} (\textbf{x}_0 - \textbf{x}_0^b) + \sum_{t=0}^{nt} (\textbf{y}_t^o - H_t(\textbf{x}_t))^T \textbf{R}_t^{-1} (\textbf{y}_t^o - H_t(\textbf{x}_t)) + \sum_{t=1}^{nt} (\textbf{x}_t - M_{(t-1) \Rightarrow t}(\textbf{x}_{t-1}))^T \textbf{Q}_t^{-1} (\textbf{x}_t - M_{(t-1) \Rightarrow t}(\textbf{x}_{t-1})) \right\} $$  

$\textbf{Q}_t$: model error (process noise) covariance at time $t$, matrix of $n \times n$  
(even given truth value of $\textbf{x}_{t-1}$, $\textbf{x}_{t}= M_{(t-1) \Rightarrow t}(\textbf{x}_{t-1})$ still has error because of imperfect model)

** Solve Weak-Constraint 4DVar**   
let：  
$\textbf{x}_0 = \textbf{x}_0^b + \textbf{E}_0 \textbf{w}_0$,  (same with *4DVar*, $\textbf{B}_0 = \textbf{E}_0 {\textbf{E}_0}^T$)   
$\textbf{x}_t = M_{(t-1) \Rightarrow t}(\textbf{x}_{t-1}) + \textbf{E}_t \textbf{w}_t$, ($t=1,\cdots,nt$, $\textbf{Q}_t = \textbf{E}_t {\textbf{E}_t}^T$)

$$\small J(\textbf{w}_0, \textbf{w}_1, \cdots) = \frac{1}{2} \left\{ \sum_{t=0}^{nt} \textbf{w}_t^T \textbf{w}_t + (\textbf{y}_0^o - H_t(\textbf{x}_0^b + \textbf{E}_0 \textbf{w}_0))^T \textbf{R}_0^{-1} (\textbf{y}_0^o - H_t(\textbf{x}_0^b + \textbf{E}_0 \textbf{w}_0)) + \sum_{t=1}^{nt} (\textbf{y}_t^o - H_t( M_{(t-1) \Rightarrow t}(\textbf{x}_{t-1}) + \textbf{E}_t \textbf{w}_t ))^T \textbf{R}_t^{-1} (\textbf{y}_t^o - H_t( M_{(t-1) \Rightarrow t}(\textbf{x}_{t-1}) + \textbf{E}_t \textbf{w}_t ))  \right\} $$   
    


**4. Hybrid 4DVar+SLAM-DA** 
$$ J(\textbf{w}_0, \textbf{x}) =  \frac{1}{2} \left\{ \textbf{w}_0^T \textbf{w}_0 + \sum_t^{nt} \sum_i^{nx} \| \frac{1}{\epsilon_x} (x_{i,t} - F(x_{i,t-1}, x_{i-1,t-1}, x_{i+1,t-1}, x_{i,t-2})) \| ^2 + \sum_t^{nt\_obs} \sum_i^{nx\_obs} \| \frac{1}{\epsilon_y} (y_{i,t} - x_{i\_obs, t\_obs}) \| ^2 \right\} $$
$$ \textrm{s.t.} \quad \textbf{x}_0 = \textbf{x}_0^b + \textbf{E}_0 \textbf{w}_0 $$

*4a. using weak constraint instead* 
$$ J(\textbf{w}_0, \textbf{x}) =  \frac{1}{2} \left\{ \textbf{w}_0^T \textbf{w}_0 + \sum_t^{nt} \sum_i^{nx} \| \frac{1}{\epsilon_x} (x_{i,t} - F(x_{i,t-1}, x_{i-1,t-1}, x_{i+1,t-1}, x_{i,t-2})) \| ^2 + \sum_t^{nt\_obs} \sum_i^{nx\_obs} \| \frac{1}{\epsilon_y} (y_{i,t} - x_{i\_obs, t\_obs}) \| ^2  +  \| \frac{1}{\epsilon_w} (\textbf{x}_0 - \textbf{E}_0 \textbf{w}_0 -  \textbf{x}_0^b) \|^2 \right\} $$
